# Stap 1 — Installeren van benodigde packages

In [1]:
!pip install openai python-dotenv pandas openpyxl requests

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
  Using cached pandas-2.3.3-cp312-cp312-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl.metadata (91 kB)
  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached numpy-2.3.5-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (62 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached et_xmlfile-2.0.0-py3-none-any.whl.metadata (2.7 kB)
Using cached pandas-2.3.3-cp312-cp312-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl (12.4 MB)
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 17.3 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


# Stap 2 — Imports en initialisatie

In [2]:
import os
from dotenv import load_dotenv
import pandas as pd
import json
import re

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise ValueError("API key not found. Voeg een .env bestand toe met OPENAI_API_KEY.")

from openai import OpenAI
client = OpenAI(api_key=OPENAI_API_KEY)

# Stap 3 — Functie om een samenvatting te genereren


In [3]:
def genereer_samenvatting(titel, eisen):
    prompt = f"""
Geef een objectieve en informatieve samenvatting van het boek met de titel:
'{titel}'.

Eisen:
- {eisen}

Gebruik geen spoilers buiten de hoofdlijn.
"""
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}]
    )

    return response.choices[0].message.content.strip()

# Stap 4 — Functie om semantische verrijking te genereren (JSON output)

In [4]:
def verrijk_semantiek(titel: str):
    prompt = f"""
Geef alleen valide JSON terug.

Analyseer het boek '{titel}' en geef het volgende terug:

- themes
- motifs
- tone
- writing_style
- character_types
- narrative_structure

Formaat:

{{
    "themes": ["..."],
    "motifs": ["..."],
    "tone": "...",
    "writing_style": "...",
    "character_types": ["..."],
    "narrative_structure": "..."
}}
"""

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "Je antwoord bevat uitsluitend JSON."},
            {"role": "user", "content": prompt}
        ]
    )

    raw = response.choices[0].message.content.strip()

    try:
        return json.loads(raw)
    except:
        pass

    try:
        json_block = re.search(r"\{.*\}", raw, re.DOTALL).group(0)
        return json.loads(json_block)
    except:
        print("Kon JSON niet parsen:")
        print(raw)
        raise

# Stap 5 — Pad en bestandsnaam opvragen


In [9]:
from pathlib import Path

bestand_pad = Path.cwd() / "test.xlsx"

if not bestand_pad.is_file():
    raise FileNotFoundError(f"Bestand niet gevonden: {bestand_pad}")

print("Bestand gevonden:", bestand_pad)


Bestand gevonden: /home/yasin/Git/Cross-domain-recommender/notebook/test.xlsx


# Stap 6 — Excel-bestand inlezen


In [13]:
from pathlib import Path
import pandas as pd

# Zoek automatisch een Excel-bestand in de huidige directory
excel_bestanden = list(Path.cwd().glob("*.xlsx"))

if not excel_bestanden:
    raise FileNotFoundError("Geen .xlsx bestanden gevonden in de huidige directory.")

bestand_pad = excel_bestanden[0]   # eerste bestand
print("Excel-bestand gevonden:", bestand_pad)

# Inlezen
df = pd.read_excel(bestand_pad)

print("\nEerste vijf rijen:")
display(df.head())

# Automatisch kiezen van verrijkbare kolom
# Regel: kies de eerste kolom waar strings in zitten
tekstkolom = None
for col in df.columns:
    if df[col].dtype == "object":    # meestal tekstkolommen
        tekstkolom = col
        break

if tekstkolom is None:
    raise ValueError("Geen tekstkolom gevonden om te verrijken.")

print("\nAutomatisch gekozen tekstkolom:", tekstkolom)


Excel-bestand gevonden: /home/yasin/Git/Cross-domain-recommender/notebook/test.xlsx

Eerste vijf rijen:


,Automerk,model
0,Opel,corsa
1,Opel,Insignia
2,Ford,focus
3,Bmw,3 series
4,BMW,5 series



Automatisch gekozen tekstkolom: Automerk


# Stap 7 — Voor elke titel samenvatting en semantische verrijking genereren


In [14]:
enriched_summaries = []
semantic_jsons = []

for idx, row in df.iterrows():
    titel = row["title"]
    eisen = "maximaal 150 woorden, objectief, geen spoilers."

    print(f"Verwerken: {titel}")

    if kolom_beschrijving and isinstance(row[kolom_beschrijving], str):
        base = row[kolom_beschrijving]
        eisen_prompt = f"Gebruik deze tekst als basis:\n{base}\n\n{eisen}"
    else:
        eisen_prompt = eisen

    summary = genereer_samenvatting(titel, eisen_prompt)
    enriched_summaries.append(summary)

    sem = verrijk_semantiek(titel)
    semantic_jsons.append(json.dumps(sem, ensure_ascii=False))

KeyError: 'title'

# Stap 8 — Verrijkte kolommen toevoegen aan dataframe


In [ ]:
df["summary_enriched"] = enriched_summaries
df["semantic_data"] = semantic_jsons

# Stap 9 — Nieuwe Excel opslaan


In [ ]:
output_path = os.path.join(map_pad, "boeken_verrijkt.xlsx")
df.to_excel(output_path, index=False)

print("Verrijkt Excel-bestand opgeslagen als:", output_path)